# Independent Example 1 - Deflections in an Annular Ring

In this problem, we solve a physically meaningful membrane deflection scenario. The domain is an annular ring, with the outer boundary fixed, and the inner boundary free. In the initial case, a uniform distributed load (UDL) acts across the membrane, this is then developed into modelling a point load. Loads are defined conventionally upwards.

We aim to compute the deflection $u(x,y)$, with outer radius $r_o$ and inner radius $r_i$, under an external pressure $p$.
\begin{align}
-T\nabla^2u = p && \mathrm{in} && \Omega = \{(x,y)\;|\; r_i \leq \sqrt{x^2 + y^2} \leq r_o\}\\
\end{align}

\begin{align}
p(x,y) = p_0\\
T(x,y) = T_0
\end{align}

## Case 1 - Thick Ring, UDL

In [1]:
import gmsh
gmsh.initialize()
gdim = 2
gmsh.model.add("annular_ring")

In [2]:
r_o = 1
r_i = 0.2
outer_disk = gmsh.model.occ.addDisk(0, 0, 0, r_o, r_o)
inner_disk = gmsh.model.occ.addDisk(0, 0, 0, r_i, r_i)
ring, _ = gmsh.model.occ.cut([(gdim, outer_disk)], [(gdim, inner_disk)])
gmsh.model.occ.synchronize()
#print(out_dim_tags)  # DEBUG: see what came out

In [3]:
membrane = ring[0][1]
pg = gmsh.model.addPhysicalGroup(gdim, [membrane], 1)

In [4]:
gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.05)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.05)
gmsh.model.mesh.generate(gdim)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 2 (Ellipse)
Info    : [ 60%] Meshing curve 3 (Ellipse)
Info    : Done meshing 1D (Wall 0.000382625s, CPU 0.000645s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0297822s, CPU 0.029307s)
Info    : 1552 nodes 3106 elements


In [5]:
from dolfinx.io import gmshio
from dolfinx.fem.petsc import LinearProblem
from mpi4py import MPI

gmsh_model_rank = 0
mesh_comm = MPI.COMM_SELF
domain, cell_markers, facet_markers = gmshio.model_to_mesh(gmsh.model, mesh_comm, gmsh_model_rank, gdim=gdim)
tdim = domain.topology.dim # element topological dimension - cells are 2D in this case
fdim = tdim - 1 # facets are therefore edges

In [6]:
from dolfinx import fem
V = fem.functionspace(domain, ("Lagrange", 1))

In [7]:
import ufl
from dolfinx import default_scalar_type
x = ufl.SpatialCoordinate(domain)
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
T = fem.Constant(domain, default_scalar_type(1))
p = fem.Constant(domain, default_scalar_type(1))

In [8]:
import numpy as np

def on_outer_boundary(x):
    return np.isclose(np.sqrt(x[0]**2 + x[1]**2), r_o)

# Locate DOF on outer boundaries
dof = fem.locate_dofs_geometrical(V, on_outer_boundary)

# Apply Dirichlet BC
bc = fem.dirichletbc(default_scalar_type(0), dof, V)

In [9]:
a = ufl.dot(T * ufl.grad(u), ufl.grad(v)) * ufl.dx
L = p * v * ufl.dx

In [10]:
from dolfinx.fem.petsc import LinearProblem
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [11]:
import pyvista
from dolfinx import plot
 
pyvista.set_jupyter_backend('trame')

domain.topology.create_connectivity(tdim, tdim)
u_top, u_cells, u_geom = plot.vtk_mesh(V)
u_grid = pyvista.UnstructuredGrid(u_top, u_cells, u_geom)

u_grid.point_data["u"] = uh.x.array
u_grid.set_active_scalars("u")

warp_grid = u_grid.warp_by_scalar("u", factor=2)
w_plotter = pyvista.Plotter()
w_plotter.add_mesh(warp_grid, show_edges=False, show_scalar_bar=True)
_ = w_plotter.screenshot("../plots/ind1-thickring.png")
w_plotter.show()

Widget(value='<iframe src="http://localhost:53005/index.html?ui=P_0x17a965a90_0&reconnect=auto" class="pyvista…

### Solution
![ThickRing](../plots/ind1-thickring.png)

The uniform load produces axisymmetric deflection with maximum displacement $(\sim0.21)$ at the free inner boundary. The smooth radial profile reflects the distributed nature of the loading, with steeper gradients near the fixed outer boundary satisfying the zero-displacement condition.

## Case 2 - Thin Ring, Point Load

To extend the problem, a more realistic loading scenario is considered - a point load. Mathematically this can be described using the Dirac delta function, characterised as below:
\begin{align}
\delta(x) = \begin{cases} 0, & \text{if } x \neq 0, \\
\infty, & \text{if } x = 0.
\end{cases} \\
\end{align}
Where $\delta(x)$ satisfies the integral:
\begin{align}
\int_{-\infty}^{\infty} \delta(x) \, dx = 1
\end{align}
As no function can satisfy the above conditions, it cannot be modelled in FEniCSx. Instead, a bivariate Gaussian with circular symmetry is used [[1]](https://archive.lib.msu.edu/crcmath/math/math/g/g078.htm):
\begin{align}
p(x, y) = \frac{1}{2 \pi \sigma^2} \exp \left(
- \frac{(x - \mu_X)^2 + (y - \mu_Y)^2}{2 \sigma^2}
\right)
\end{align}

$f(x,y)$ can be scaled by $A$ to provide the total integrated load strength, and $\mu_X$ and $\mu_Y$ represent the location of the peak load, $\left(x_0,y_0\right)$. Thus we have:

\begin{align}
-T\nabla^2u = \frac{A}{2 \pi \sigma^2} \exp \left(
- \frac{1}{2\sigma^2}\left((x - x_0)^2 + (y - y_0)^2\right)
\right)
\end{align}

Scaling this equation [[2]](https://jsdokken.com/dolfinx-tutorial/chapter1/membrane.html) collects terms of the same physical quantity. $D_e$ is the characteristic deflection. Starting with $R = R_o$,

\begin{align}
\bar{x} = \frac{x}{R}, && \bar{y} = \frac{y}{R}, && \bar{w} = \frac{u}{D_e}, &&
\bar{x_0} = \frac{x_0}{R}, && \bar{y_0} = \frac{y_0}{R} \\
\end{align}

\begin{align}
w\left(\bar{x},\bar{y}\right) = \frac{u\left(x,y\right)}{D_e}
\end{align}

\begin{align}
\bar{\Omega} = \{\rho \in \left(\rho_{in},1 \right)\}, && \rho := \frac{R_i}{R_o}
\end{align}

Taking the Laplacian of $w\left(\bar{x},\bar{y}\right)$ allows us to reformulate the PDE. Firstly, using the chain rule:

\begin{align}
-T\nabla^2u = p\\
\end{align}

\begin{align}
-T\left(\frac{\partial^2u}{\partial x^2} + \frac{\partial^2u}{\partial y^2} \right) = p(x,y)
\end{align}

\begin{align}
-T\left[\frac{D_e}{R^2} \left(\frac{\partial^2w}{\partial \bar{x}^2} + \frac{\partial^2w}{\partial \bar{y}^2} \right)\right] = p(R\bar{x},R\bar{y})
\end{align}

\begin{align}
-T\left[\frac{D_e}{R^2} \left(\frac{\partial^2w}{\partial \bar{x}^2} + \frac{\partial^2w}{\partial \bar{y}^2} \right)\right] = \frac{A}{2 \pi \sigma^2} \exp \left(
- \frac{1}{2\sigma^2}\left((R\bar{x} - x_0)^2 + (R\bar{y} - y_0)^2\right)
\right)
\end{align}

Thus we have:

\begin{align}
-\nabla^2w = \alpha  \exp \left( -\beta^2 \left[ \left( \bar{x} - \bar{x_0} \right)^2 + \left( \bar{y} - \bar{y_0} \right)^2
 \right]
\right)
\end{align}

for:

\begin{align}
\alpha = \frac{R^2A}{2 \pi \sigma^2 TD_e}, && \beta^2 = \frac{R^2}{2\sigma^2}
\end{align}

We work in dimensionless units with $T_0=1$ and choose $D_e=1$ as our characteristic deflection scale. This choice, combined with our selection of $A=1$ for the integrated load strength, ensures all terms in the non-dimensional equation are $\sim O(1)$.

\begin{align}
-\nabla^2w \sim O(1)
\end{align}

In [12]:
gdim = 2
gmsh.model.add("thin_ring") # prefix t_~ refers to the thin domain
t_r_o = 1
t_r_i = 0.5
t_outer_disk = gmsh.model.occ.addDisk(0, 0, 0, t_r_o, t_r_o)
t_inner_disk = gmsh.model.occ.addDisk(0, 0, 0, t_r_i, t_r_i)
t_ring, _ = gmsh.model.occ.cut([(gdim, t_outer_disk)], [(gdim, t_inner_disk)])
gmsh.model.occ.synchronize()

t_membrane = t_ring[0][1]
pg2 = gmsh.model.addPhysicalGroup(gdim, [t_membrane], 1)

gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.05)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.05)
gmsh.model.mesh.generate(gdim)

from dolfinx.io import gmshio
from dolfinx.fem.petsc import LinearProblem
from mpi4py import MPI

gmsh_model_rank = 0
mesh_comm = MPI.COMM_SELF
t_domain, cell_markers, facet_markers = gmshio.model_to_mesh(gmsh.model, mesh_comm, gmsh_model_rank, gdim=gdim)
tdim = t_domain.topology.dim # element topological dimension - cells are 2D in this case
fdim = tdim - 1 # facets are therefore edges

from dolfinx import fem
V = fem.functionspace(t_domain, ("Lagrange", 1))

Info    : Meshing 1D...                                                                                                                        
Info    : [  0%] Meshing curve 2 (Ellipse)
Info    : [ 60%] Meshing curve 3 (Ellipse)
Info    : Done meshing 1D (Wall 9.2e-05s, CPU 0.000234s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0128872s, CPU 0.020942s)
Info    : 1247 nodes 2496 elements


To maintain total source strength $ I_\Omega \sim O(1)$, we must scale $\alpha$ with $\beta$:

\begin{align}
\iint_\Omega \alpha  \exp \left( -\beta^2 \left[ \left( \bar{x} - \bar{x_0} \right)^2 + \left( \bar{y} - \bar{y_0} \right)^2
 \right]
\right) d\bar{x} d\bar{y} = A \sim O(1)
\end{align}

Defining $\chi = \bar{x} - x_0$ and $\zeta = \bar{y} - y_0$, we have:
\begin{align}
d\bar{x} d\bar{y} = d\chi d\zeta
\end{align}

\begin{align}
\Omega' = \{ \left(\chi,\zeta \right) \;|\; r_i \leq \sqrt{\left(\chi+x_0\right)^2 + \left(\zeta+y_0\right)^2} \leq r_o\ \}
\end{align}

\begin{align}
A_{\Omega'} = \iint_{\Omega'} \alpha  \exp \left( -\beta^2 \left[ \chi^2 + \zeta^2 \right]\right) d\chi d\zeta \sim O(1)
\end{align}

When $\beta$ is sufficiently large, such that the Gaussian is confined within the domain, we can use a standard solution as below. The co-efficient 3 is derived from the strength of the Gaussian - 99.7% is contained within 3$\sigma$. The integral limit to infinity is valid for a rapidly decaying Gaussian.

\begin{align}
\Omega' \supseteq B\left(\left(x_0,y_0\right), \frac{3}{\beta}\right)
\end{align}

\begin{align}
\iint_{\mathbb{R^2}} \exp \left( -\beta^2 \left[ \chi^2 + \zeta^2 \right]\right) d\chi d\zeta = \frac{\pi}{\beta^2}
\end{align}

Therefore we have, for $T, D_e \sim O(1)$:
\begin{align}
A_{\Omega'} \approx \alpha \cdot \frac{\pi}{\beta^2}
\end{align}

And so:
\begin{align}
\alpha \approx \frac{A_{\Omega'}\beta^2}{\pi}, && \mathrm{if} &&r_o - r_i \geq \frac{3}{\beta} 
\end{align}

We can choose a scalar $>3$, in this case $5$, to ensure compliance.

In [13]:
import ufl
from dolfinx import default_scalar_type

import numpy as np

def on_outer_boundary(x):
    return np.isclose(np.sqrt(x[0]**2 + x[1]**2), t_r_o)

dof = fem.locate_dofs_geometrical(V, on_outer_boundary)

# Apply Dirichlet BC
bc = fem.dirichletbc(default_scalar_type(0), dof, V)

In [14]:
import ufl
from dolfinx import default_scalar_type
x = ufl.SpatialCoordinate(t_domain)
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

A = 1 #Apply load ~ O(1)
beta = 5/(t_r_o-t_r_i)
alpha = A * beta**2 / np.pi
x_0 = fem.Constant(t_domain, default_scalar_type(0.53))
y_0 = fem.Constant(t_domain, default_scalar_type(0.53))
p_point = alpha*ufl.exp(-beta**2 * ((x[0] - x_0)**2 + (x[1] - y_0)**2))


a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = p_point * v * ufl.dx

In [15]:
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
t_uh = problem.solve()

In [16]:
Q = fem.functionspace(t_domain, ("Lagrange", 4))
expr = fem.Expression(p_point, Q.element.interpolation_points())
pressure = fem.Function(Q)
pressure.interpolate(expr)

In [17]:
from dolfinx.plot import vtk_mesh
import pyvista
pyvista.set_jupyter_backend('trame')

# Extract topology from mesh and create pyvista mesh
topology, cell_types, x = vtk_mesh(V)
grid = pyvista.UnstructuredGrid(topology, cell_types, x)

# Set deflection values and add it to plotter
grid.point_data["u"] = t_uh.x.array
warped = grid.warp_by_scalar("u", factor=2)

plotter = pyvista.Plotter()
plotter.add_mesh(warped, show_edges=False, show_scalar_bar=True, scalars="u")
_ = plotter.screenshot("../plots/ind1-thinring.png")
plotter.show()

Widget(value='<iframe src="http://localhost:53005/index.html?ui=P_0x315f78550_1&reconnect=auto" class="pyvista…

### Solution
![ThinRing](../plots/ind1-thinring.png)

The concentrated load creates a localized deflection peak $(\sim0.31)$ near the load center at $(0.53,0.53)$. The rapid spatial decay demonstrates membrane tension redistributing the concentrated force across the domain. The Gaussian approximation width $(\sigma \approx 0.14)$ smooths the theoretical singularity of a true point load.